# Compute GLORYs EOFs
## Created by Dani Lafarga 5/28/2025
## Last updated on: 8/14/2025
This program will compute 3D EOFs from GLORYS temperature data found at:  https://data.marine.copernicus.eu/product/GLOBAL_MULTIYEAR_PHY_001_030/description

The program will partition the data according to the total amount of GB specified by the user to compute both covariance and 3D EOFs. 

Program will output to a specified directory called EOF_directory:
- eigenvalues/eigenvectors in a CSV file
- EOFs in seperate NetCDF files according to mode

This version assumes the user will be able to load one year's worth of anomaly data. 

Code is split up into 4 sections:
- Section 1: Defining all directories and dimension variables
- Section 2: Compute Covariance using PIM
- Section 3: Compute eigenvalues/eigenvectors
- Section 4: Compute EOFs in parts

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/dlafarga/PIM-for-Computing-3D-EOFs/blob/main/3D%20EOFs%20GLORYS.ipynb)

In [2]:
import numpy as np
from numpy import meshgrid
import scipy.io as sc
import os
from pprint import pprint
import matplotlib.pyplot as plt
from numpy import linspace
from numpy import meshgrid

import matplotlib as mpl
import matplotlib
import math
import time

import netCDF4 as nc 
from netCDF4 import Dataset as ds


import warnings
warnings.filterwarnings("ignore")

#################################################################################################################
#################################################################################################################

# Function creates a folder if it doesnt already exist
# Input:
#         - folder_path: string with the path name to folder
def create_folder(folder_path):
    
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"Folder '{folder_path}' created.")
    else:
        print(f"Folder '{folder_path}' already exists.")
     
#################################################################################################################
#################################################################################################################
# Function converts longitude from 0 to 360 to centered at 180. This is just for naming purposes.
# it does not shift the axis of the data
# Input:
#         - lon_0_360: float with the longitude value from 0 to 360
# Output:
#         - convert: String with the longitude value on a 180 centered scale

# Important Variables:
#         - lon_180_w_180_e: float with the longitude value on a 180 centered scale
def convert_longitude(lon_0_360):
    lon_180_w_180_e = lon_0_360 - 360 if lon_0_360 > 180 else lon_0_360
    if lon_180_w_180_e < 0:
        lon_180_w_180_e = abs(lon_180_w_180_e)
        convert = f"{lon_180_w_180_e}$^\circ$W"
    else:
        convert = f"{lon_180_w_180_e}$^\circ$E"
    return convert

# Section 1: Defining ALL directories and dimension variables
## Dimension variabels:
- lat: latitude from raw data
- lon: longitude from raw data
- depths: depth from raw data
- years: defined years
- cut_lat: cut lattitude values based on lat_start and lat_end
- cut_lon: cut longitude values based on lon_start and lon_end
- cut_depths: cut depth values based on depth_end

## Directories that must be changed by user:
- data_directory: This will be the folder you will be reading the raw data from. This is used mainly to define the latitude, longitude, and depths if they arent already saved in the anomalies
- anomalies_directory: This is the folder that contains all anomalies. **Code will need already computed anomalies** if you haven't already done so please compute them using the compute anomalies and climatologies code.
- EOF_directory: this will be where we save all EOFs and eigenvalues/eigenvectors. This folder will be created for you in the first section
  
## Variables that need to be changed by user
- years: please set this to the starts and end year of the anomalies
- lat_start: Starting latitude index. If entire region use 0
- lat_end: End latitude index. If entire region use len(lat)
- lon_start: Starting longitude index. If entire region use 0
- lon_end: End longitude index. If entire region use len(lon)

In [9]:
#################################################################################################################
#################################################################################################################
# Function get_var() will get common variables that will be required for climatologies,
# anomalies, and EOFs
# Input: 
#         - MON_INDEX: month index tells us home many years we will have 
#              Months 1-6 have 29 years
#              Months 7-12 only have 28 years
#              MON_INDEX = 0 means 3 month season
# Output: 
#         - lat: 1d array with all latitude values
#         - lon: 1d array with all longitude values
#         - depth: 1d array with all depth values
#         - years: 1d array with all year values
def get_var():
    fn     = 'thetao.mon.mean.1993.nc'
    fn     = os.path.join(data_directory, fn)
    fn     =  ds(fn,'r')
    lat    = fn.variables['latitude'][:].data    # read in latitude
    lon    = fn.variables['longitude'][:].data   # read in longitude
    depths = fn.variables['depth'][:].data       # read in depth
    fn.close()
    return lat, lon, depths

# Please change the following directories
data_directory          = 'E:/GLORYS/'
anomalies_directory     = os.path.join(data_directory, 'Anomalies/Winter Avg' )
EOF_directory           = os.path.join(data_directory, 'cut_EOFs/Entire Ocean PIM')

create_folder(EOF_directory) # if directory doesnt exist create it


Folder 'E:/GLORYS/cut_EOFs/Entire Ocean PIM' already exists.


In [11]:
lat, lon, depths = get_var() # get latitude, longitude, and latitude variables


'''
If you would like to do a regional calculation consider the following indices

depth_end = 32 # first 500 m

# for the entire Pacific
lat_start = 720
lat_end   = 1801
lon_start = 1440
lon_end   = 3600
depth_end = 32


# for ENSO
lat_start = 600
lat_end   = 1321
lon_start = 1320
lon_end   = 3481
depth_end = 32


# Global
lat_start = 0
lat_end   = len(lat)
lon_start = 0
lon_end   = len(lon)
depth_end = len(depths)
'''


lat_start = 0
lat_end   = len(lat)
lon_start = 0
lon_end   = len(lon)
depth_end = len(depths)

years  = np.linspace(1994, 2021,28, dtype="int")

cut_lat = lat[lat_start:lat_end]
cut_lon = lon[lon_start:lon_end]
cut_depths = depths[:depth_end]

# Section 2: Compute Covariance
**Please be sure file names line up**

In [12]:
#################################################################################################################
#################################################################################################################
# Function reads in cut anomalies, will take out NaNs, and then flattten to a 1D array
# 
# Input:
#         - year: int with the year to read in
# Output:
#         - anom: 1d array that contains anomalies of the file without any NaN
def read_anom_compressed(year):
    fn   = 'anom.mon.'+str(year)+'.nc'                 # change file name
    fn   = os.path.join(anomalies_directory, fn)       # join with the directory that can be changed
    nc0  = ds(fn, 'r')
    anom = nc0.variables['anom'][:depth_end, lat_start:lat_end, lon_start:lon_end]
    if anom.shape[2] == len(lon):                  # for full ocean there is a mismatch in GLORYS
        anom.mask[13, 969, 3717] = True
    anom = anom.compressed()                           # flatttentens and gets rid of NaN
    nc0.close()
    return anom
#################################################################################################################
#################################################################################################################
# Function reads in cut anomalies for multiple years, and will put them all into a matrix
# 
# Input:
#         - year_array: 1D array with mutliple years. Range will be from original year array 
# Output:
#         - anom_mat: 2D array where rows are data points, and columns are the years defined by
#                     the year array.  Array does not contain Nan (land points)
def read_anom_compressed_matrix(year_array):
    anom_mat = []                         # initialize array matrix as a list
    for year in year_array:               # read in anom for every year
        anom = read_anom_compressed(year) # read in anomaly with no NaN
        anom_mat.append(anom)             # add to the array matrix 
    anom_mat = np.array(anom_mat)         # turn anom into an array
    return anom_mat
#################################################################################################################
#################################################################################################################
# function will compute how many years could be read in at a time for matrix multiplication
# input: 
#       - GB_avail: int taken as input from user

# output:
#       - years_possible: int that will define how many years can be read in each year for
#                         the matrix multiplication in the covariance calculation
# important variables:
#       - anom: a compressed 1D array with anomalies and no NaN
def years_possible_for_cov(GB_avail):
    anom = read_anom_compressed(1994) # reading in one year of data with no NaN
    if GB_avail*1e9 < 2*anom.nbytes:  # checking if one year is possible for the calculation
        years_possible = 1
        print(f"Data exceeds the inputed memory try reading in one year at a time.")
        print(f"One year will require a total of {anom.nbytes*2} bytes. If this is too much try a smaller region.")
        return years_possible
    length_test = (GB_avail*1e9)/(anom.nbytes)    # divide the GB by bytes to convert to enteries
    years_possible = 1                            # initialize total years possible
    while 1:                                      # finding greatest integer divisor
        if length_test/(2*years_possible) > 1:
            years_possible = years_possible + 1
            continue;
        else:
            years_possible = years_possible-1
            break;
    if years_possible > len(years):              # shouldn't be more than the total amount of years
        years_possible = len(years)
    print(f"With {GB_avail} GB you will be able to read in {years_possible} years at a time")
    total_GB = (years_possible * 2) * anom.nbytes
    total_GB = total_GB / 1e9
    print(f"{years_possible} years will require {total_GB} GB")
    return np.int(years_possible)
#################################################################################################################
#################################################################################################################
# function will calculate how many GB memory it will take to calculate the covariance matrix
# a specified amount of years 
# input:
#     - year: int with however many years
# output: will print total GB required
def year_GB_calc(year):
    if year > len(years): # should not excede the total amount of years we have
        print(f"That's more years than we have. Did you mean {len(years)}? Try again with that many.")
        return
    # calculating the total GB
    anom = read_anom_compressed(1994)
    total_GB = (year * 2) * anom.nbytes
    total_GB = total_GB / 1e9
    print(f"{year} years will take {total_GB} GB")   
#################################################################################################################
#################################################################################################################
# Function computes the covariance matrix in time for the data. 
# Covariance is computed by doing the dot product of one anomally array at time t
# with all other anomalies at time t'. 
# to avoid redundancies the code will explot the symmetry of the covariance matrix
'''
Example
   Anomaly Mult: 
            let t = 2
                   o t' = t+1
                                        anom(t) * anom(t+1) = a
                                        -> cov(2,3) = cov(3,2) = a
                   o t' = t+2
                                        anom(t) * anom(t+2) = b
                                        -> cov(2,4) = cov(4,2) = b
                   o t' = t+3
                                        anom(t) * anom(t+3) = c
                                        -> cov(2,5) = cov(5,2) = c
                                                ┌               ┐
                                        cov =   │  # # # # # #  │
                                                │  # # # # # #  │
                                                │  # # t a b c  │
                                                │  # # a # # #  │
                                                │  # # b # # #  │
                                                │  # # c # # #  │
                                                └               ┘
For multiple years at once a , b, and c are 1D arrrays input into the covariance matrix
'''

#################################################################################################################
#################################################################################################################
# Function computes the covariance matrix in time for the data. 
# Covariance is computed by doing the dot product of one anomally array at time t
# with all other anomalies at time t'. 
# to avoid redundancies the code will explot the symmetry of the covariance matrix
# this function is derivative of the previous cov() function
# Input:
#         - years: 1D array of ints with all years to be used in computation. These are all 
#                  the anomalies which will be read in 
#         - years_possible: the total amount of years to read in per iteration based on the specific amount of memory
#                  the user wants to use. 
# Output:
#         - covariance_mat: 2D covariance matrix(floats)
# Important Variables:   
#         - Y: total amount of years. This will give us the dimensions for our square covariance matrix (Y by Y)
#         - t: the time index(int). this will tell us what diagonal we are on so we can exploit symmetry.
#              It will advance by years_possible (ex. years_possible = 3, t = 0,3,6...)
#         - anom: a 2D N' by years_possible matrix that contain anomalies starting at time years[t]. 
#         - t_prime: all values from t+1 to the last time step index(int). This will advance by years_possible just
#                  like t
#         - year_array: the year aray value starting at t
#         - year_array_prime: the year aray value starting at t_prime
'''
             example: years = [1993 to 2021], if t = 3 and years_possible = 3
                     
                     year_array = [1995, 1996, 1997]
                     so...
                     t_prime = 3 to 29 nd increases by 3
                     
                     at t_prime = 3
                        -> year_array_prime = [1995, 1996, 1997]
                        
                     at t_prime = 6
                        -> year_array_prime = [1998, 1999, 2000]
                        
'''
#         - anom_tprime: a 2D N' by years_possible matrix that contain anomalies starting at time years[t]. 
#         - resulting_matrix_mult: result of anom dotted with anom_tprime. This makes it easier to input into covariance matrix
#                                 the result will be a years_possible by years_possible matrix
def cov_mat(years, years_possible):
    start = time.time()
    Y = len(years)
    covariance_mat = np.zeros((Y,Y))
    for t in range(0,Y,years_possible):
        year_array = years[t:t+years_possible]           # read in the array of years
        anom = read_anom_compressed_matrix(year_array)   # read in anomalies for years specified 
        for t_prime in range(t,Y,years_possible):      
            year_array_prime = years[t_prime:t_prime+years_possible]   # read in the array of years
            anom_tprime = read_anom_compressed_matrix(year_array_prime)# read in anomalies for years specified 
            resulting_matrix_mult =  np.dot(anom, anom_tprime.T)       # do dot product for covariance
            covariance_mat[t:t+years_possible, t_prime: t_prime+len(year_array_prime)] = resulting_matrix_mult   # input to covariance matrix
            covariance_mat[t_prime: t_prime+len(year_array_prime),t:t+years_possible]  = resulting_matrix_mult.T # input the reflected side
    end = time.time()
    time_total = end-start
    print(time_total)
    return covariance_mat

In [27]:
# calculating years possible according to specified memory
GB_avail = int(input("How Many GB would you like to use?:"))
years_possible = years_possible_for_cov( GB_avail)

How Many GB would you like to use?: 5


With 5 GB you will be able to read in 2 years at a time
2 years will require 4.118381888 GB


In [34]:
# how many GB for the amount of years
years_possible = int(input("How many years would you like to read?:"))
year_GB_calc(years_possible)

How many years would you like to read?: 3


3 years will take 6.177572832 GB


In [58]:
cov = cov_mat(years, years_possible)

2049.655123233795


# Section 3: Compute eigenvalues/eigenvectors
compute eigenvectors and eigenvalues from the temporal covariance matrix.
$$
 C_T v_k = \lambda_k v_k
$$

where:
- $C_S$ : temporal covariance
- $v_k$ : eigenvectors. These are also pricipal components (PC)
- $\lambda_K$ : eigenvalues. These also give associated variability
- $k$ : the mode.The total amount of these will depend on the dimensions of the covariance matrix.

In [84]:
import csv
#################################################################################################################
#################################################################################################################
# Function computes eigenvectors and values of the covariance matrix and sort them by variance greatest to least
# function will also check to make sure they are normalized and orthogonal
# Input:
#         - cov: 2D covariance matrix. Cannot have any NaNs!
# Output:
#         - eigvals: 1D array with all eigenvalues of covariance matrix sorted (float)
#         - eigvecs: 2D matrix with corresponding sorted eigenvectors of covariance matrix (float)
# Important Variables: 
def find_eigen(cov):
    eigvals, eigvecs = np.linalg.eig(cov)
    eigvecs = np.array(eigvecs)
    
    # Check to make sure vectors are orthogonal
    n, m = eigvecs.shape
    for col_i in range(m):
        for col_j in range(m):
            if col_i < col_j:  # use strictly less than because we don't want to consider a column with itself, and the dot product is commutable so order doesn't matter
                is_orthogonal = np.dot(eigvecs[:, col_i], eigvecs[:, col_j])
                if not np.isclose(is_orthogonal, 0):
                    raise ValueError(f"Eigenvector {col_i} and Eigenvector {col_j} are not orthogonal.")
    # checking magnitude of evals
    for col_i in range(m):
        is_one = np.linalg.norm(eigvecs[:,col_i])
        if not np.isclose(is_one, 1):
            raise ValueError(f"Eigenvector {col_i} is not one")
            
    # sort eigenvalues and eigenvectors
    eig_index = np.argsort(eigvals)
    eig_index = np.flip(eig_index)
    
    new_eigvals = eigvals[eig_index[:]]
    eigvals = new_eigvals
    
    new_eigvecs = eigvecs[:,eig_index]
    eigvecs = new_eigvecs 
    return eigvals,eigvecs
#################################################################################################################
#################################################################################################################
# Function saves eigenvalues and eigencvectors into a csv file 
# 
# Input:
#         - eigvals: 1D array with eigenvalues
#         - eigvecs: 2D matrix with eigenvectors
#         - month: string with the month the eigenvec/eigenvals belong to
# Output:
#         - None: function saves to a file 
def save_evals(eigvals,eigvecs,month):
    # Prepare data for saving
    # Create a 2D array with eigenvalues and eigenvectors
    data = np.hstack((eigvals.reshape(-1, 1), eigvecs.T))
    
    # Save to CSV file
    create_folder(EOF_directory)
    fn     = month+'_eigenvalues_eigenvectors.csv'
    fn     = os.path.join(EOF_directory, fn)
    with open(fn, 'w', newline='') as file:
        writer = csv.writer(file)
        
        # Write the header
        header = ['Eigenvalue'] + [f'Eigenvector {i+1}' for i in range(eigvecs.shape[1])]
        writer.writerow(header)
        
        # Write the data
        for row in data:
            writer.writerow(row)
    
    print("Eigenvalues and eigenvectors saved to 'eigenvalues_eigenvectors.csv'")

In [990]:
eigvals,eigvecs = find_eigen(cov)
save_evals(eigvals,eigvecs,'Winter Avg')

Folder 'E:/GLORYS/cut_EOFs/Entire Ocean PIM' already exists.
Eigenvalues and eigenvectors saved to 'eigenvalues_eigenvectors.csv'


# Section 4: Compute EOFs in parts

### Space time anomaly Matrix:
Let an eigenvector be defined as $\vec{v_k}$ where $k$ is the eigenvector number or mode number. There are a total of Y modes that are used. Each EOF $u_k$ can be computed by the projection of anomalies $A$ onto $v_k$ in the following way:
$$
\begin{equation}
    u_k = \frac{Av_k}{|Av_k|},
\end{equation}
$$

where $|Av_k|$ is the Euclidean norm of the vector $Av_k$.

This algorithm breaks up this computation so that it reads anomalies a chunk at a time and writes to EOF files a chunk at a time. Algorithm will skip over any row that is NaN which is consistent for every year. Meaning it will neither read nor write to those rows.

In [86]:
import pandas as pd
#################################################################################################################
#################################################################################################################
# Function will read in eigenvalues and eigenvectors saved in CSV file based on the EOF_directory
# Input: None
# Output:
#         - eigvals: a 1D float array with all eigenvectors corresponding eigenvalues
#         - eigvecs: a 2D float array with all eigenvectors in the columns of the matrix

# Important Variables:
#         - month: defined previously by the Set_Global_Variables function 
#         - EOf_directory: defined by the set_EOF_Anom_directory function 
def read_evec(prefix):
    # Read the CSV file
    fn     = prefix +'_eigenvalues_eigenvectors.csv'
    fn     = os.path.join(EOF_directory, fn)
    df = pd.read_csv(fn)

    # Extract the eigenvalues
    eigvals = df['Eigenvalue'].values

    # Extract the eigenvectors
    eigvecs = df.drop(columns=['Eigenvalue']).values.T
    return eigvals,eigvecs

In [88]:
_,eigvecs = read_evec('Winter Avg')
eigvecs = eigvecs.astype(np.float32) # making sure eigenvectors are of the same type

In [90]:
#################################################################################################################
#################################################################################################################
# Function checks if a file exists and if it does, continues to the next iteration.
# Input:
#         - file_path: string with the path name to folder
# Outputs:
#         - True if file doesnt exists
#         - False if file exists
# Important Variables:
def check_file_and_continue(file_path):
    if os.path.exists(file_path):
        print(f"File '{file_path}' exists. Moving to next iteration.")
        return False
    else:
        return True
#################################################################################################################
#################################################################################################################
# Function reads in raw cut anomalies
#
# Input:
#         - year: int with the specific year to read
# Output:
#         - anom: 3D array of dim (depth, lat, lon) with the anomaly data
def read_anom_raw(year):
    fn   = 'anom.mon.'+str(year)+'.nc'            # change file name
    fn   = os.path.join(anomalies_directory, fn)  # join with the directory that can be changed
    nc0 = ds(fn, 'r')
    anom = nc0.variables['anom'][:depth_end, lat_start:lat_end, lon_start:lon_end]
    if anom.shape[2] == len(lon):                 # for full ocean there is a mismatch
        anom.mask[13, 969, 3717] = True
    anom = anom.filled()
    nc0.close()
    return anom
#################################################################################################################
#################################################################################################################
# Function reads anomaly data but only outputs matrix with no NaNs
# depth is specified to save RAM while reading. Without the specified depth you would be reading in all depths
# Input:
#         - year: int with the specific year to read
#         - read: 2D array with indexes of what page (depth), row, and column has data.
#                 read[:,0]- the page (depth) with data
#                 read[:,1]- the rows with data
#                 read[:,1]- the columns with data
# Output:
#         - anom: 2D matrix at specified year and depth with no NaNs

def read_anom_parts(year,read):
    anom = read_anom_raw(year)
    anom = anom[read[:,0], read[:,1], read[:,2]]
    return anom
#################################################################################################################
#################################################################################################################
# Function Created the files which will be aggregated a chunk at a time.
# The files must be created  first so we can more easily access and aggregate them.
# Input:
#         - month: string  where to save the EOFs
#         - total_modes: int with the total amount of modes I want.
#                        Maximum is the total amount of time steps
# Output:
#         - None: the function will create files for all EOF modes
# Important Variables:
def create_EOF_files(total_modes):
    create_folder(EOF_directory)                  # create folder named by the month
    anom_base = read_anom_raw(1998)               # read in one anomaly to base the dimensions of EOF
    EOF_mat = np.full_like(anom_base, np.nan)     # create a matrix with the same dim size as the anomalies
    for mode in range(1,total_modes+1):           # for all the modes
        # Create a new NetCDF file
        fn = 'EOF_'+str(mode)+'.nc'               # file name for EOFs
        fn     = os.path.join(EOF_directory, fn)  # join with EOF path
        if not check_file_and_continue(fn):                      # if the file does already exist continue
            continue;
        ncfile = ds(fn, 'w', format='NETCDF4')                   # if the file doesnt exist create it
        
        # Define dimensions
        ncfile.createDimension('depth', EOF_mat.shape[0])
        ncfile.createDimension('lat', EOF_mat.shape[1])
        ncfile.createDimension('lon', EOF_mat.shape[2])

        # Create a variable for EOFs
        lat_save = ncfile.createVariable('latitude', 'float32', ('lat'), fill_value = np.nan)
        lon_save = ncfile.createVariable('longitude', 'float32', ('lon'), fill_value = np.nan)
        depth_save   = ncfile.createVariable('depth', 'float32', ('depth'), fill_value = np.nan)
        var = ncfile.createVariable('EOF', 'f4', ('depth', 'lat', 'lon'), fill_value = np.nan)
        
        # saving variables
        lat_save[:]  = cut_lat
        lon_save[:]  = cut_lon
        depth_save[:]    = cut_depths
        
        var[:] = EOF_mat # save the empty EOF
        ncfile.close()   # close the file
    print(f"Total of '{mode}' EOF files created or ready to save to.")
#################################################################################################################
#################################################################################################################
# function will return the total amount of enteries that can be read in for every year
# given the total GB availiable
# input:
#     - GB_avil: int that decribes the total GB that is avaliable to use
# ouput:
#     - chunk_size: the total amount of enteries read in every year for the EOF PIM calculation
def Chunk_size_EOF_calc(GB_avail):
    # calculating the total GB
    anom = read_anom_compressed(1994)
    chunk_size = int((GB_avail*1e9 - anom.nbytes)/4 * 1/(len(years)))
    print(f"You can read in {chunk_size} enteries every year.")  
    return chunk_size
#################################################################################################################
#################################################################################################################
# Function computes EOFs in parts.
# To save on RAM we exclude points with NaN when reading
# Input:
#         - years: 1D int array with all years to read in of the anomalies.
#                  Length of this will also be the max amount of modes
# Output:
#         - None: function saves all EOF modes in sperate files
# Important Variables:
#         - total_modes: int of the total amount of modes to compute. The default is the max (see input for max)
#         - num_ind: 2D array that tells us which indexes have ocean data(excludes land/NaN).
#         - chunk_size: the total amount of grid points to read in each year. Remember we are excluding NaN so
#                       this is total grid points with data that we are reading in
#         - read: a subset of num_ind that contains a total of chunk_size indexes for that iteration.
#                 This variable makes sure we are not reading the same chunk, and the for loop it is in makes
#                 sure we read to the end even if the last chunk is less than chunk_size
#         - anom_mat: 2D space time anomaly matrix without NaN. This is at most dim(chunk_size, Y)
#         - EOF_mat: 3D Array of size (depth, lat, lon). This contains the resulting EOFs from the
#                    multiplication.
#                   NOTE: this matrix does contain NaNs. We make sure that the resulting multiplacation goes
#                         to the correct index bny uising the variable read for the indexing.
def compute_EOFs(years, chunk_size):
# Compute EOFs over all depths at once by chunking through the list of (depth, lat, lon) locations where anom_base is not NaN.
    start = time.time()                            # for timing
    total_modes = len(years)+1                     # number of EOF modes to compute

    # read one year just to know the shape and mask
    anom_base = read_anom_raw(1994)                # shape (depth, lat, lon)

    # get all non-NaN grid-points (depth, lat, lon)
    num_ind = np.argwhere(~np.isnan(anom_base))    # shape (N, 3)

    for j in range(0, num_ind.shape[0], chunk_size):
        read = num_ind[j:j+chunk_size]             # shape (M, 3)
        anom_mat = np.zeros((read.shape[0], len(years)), dtype=np.float32)
        for k, y in enumerate(years):
            # read the full 3D anomalies for year y
            anom_mat[:, k] = read_anom_parts(y, read)  # (depth, lat, lon)

        # for each EOF mode, write the computed values
        for mode in range(1, total_modes):
            fn = 'EOF_'+str(mode)+'.nc'
            fn = os.path.join(EOF_directory, fn)
            EOF_ncfile = ds(fn, 'a')
            EOF = EOF_ncfile.variables['EOF']
            EOF_mat = EOF[:].filled()
            # compute EOFs and write to specified lat, lon
            EOF_mat[read[:,0], read[:,1], read[:,2]] = anom_mat @ eigvecs[:, mode-1]
            EOF[:] = EOF_mat
            EOF_ncfile.close()

    # normalize_EOFs(len(years))
    end = time.time()
    time_total = end - start
    print(time_total)
    return 
#################################################################################################################
#################################################################################################################
# Function normalizes the EOFs by dividing by magnitude
#
# Input:
#         - tot_modes: an int describing total amount of modes
# Output:
#         - None: saves the EOFs to the EOF files
# Important Variables:
def normalize_EOFs(tot_modes):
    for mode in range(1,tot_modes+1):
        fn      = 'EOF_'+str(mode)+'.nc'
        fn      = os.path.join(EOF_directory, fn)     # join with EOF path
        EOF_ncfile = ds(fn, 'a')
        EOF     = EOF_ncfile.variables['EOF']
        EOF_mag = EOF[:].compressed()                      # flattens and gets rid of NaN
        EOF[:]  = EOF[:].filled()/np.linalg.norm(EOF_mag)  # divide by magnitude
        EOF_ncfile.close()

In [20]:
create_EOF_files(len(years))

Folder 'E:/GLORYS/cut_EOFs/Entire Ocean PIM' already exists.
File 'E:/GLORYS/cut_EOFs/Entire Ocean PIM\EOF_1.nc' exists. Moving to next iteration.
File 'E:/GLORYS/cut_EOFs/Entire Ocean PIM\EOF_2.nc' exists. Moving to next iteration.
File 'E:/GLORYS/cut_EOFs/Entire Ocean PIM\EOF_3.nc' exists. Moving to next iteration.
File 'E:/GLORYS/cut_EOFs/Entire Ocean PIM\EOF_4.nc' exists. Moving to next iteration.
File 'E:/GLORYS/cut_EOFs/Entire Ocean PIM\EOF_5.nc' exists. Moving to next iteration.
File 'E:/GLORYS/cut_EOFs/Entire Ocean PIM\EOF_6.nc' exists. Moving to next iteration.
File 'E:/GLORYS/cut_EOFs/Entire Ocean PIM\EOF_7.nc' exists. Moving to next iteration.
File 'E:/GLORYS/cut_EOFs/Entire Ocean PIM\EOF_8.nc' exists. Moving to next iteration.
File 'E:/GLORYS/cut_EOFs/Entire Ocean PIM\EOF_9.nc' exists. Moving to next iteration.
File 'E:/GLORYS/cut_EOFs/Entire Ocean PIM\EOF_10.nc' exists. Moving to next iteration.
File 'E:/GLORYS/cut_EOFs/Entire Ocean PIM\EOF_11.nc' exists. Moving to next it

In [ ]:
# input how much GB you wish to use
GB_avail = int(input("How Many GB would you like to use?:"))
chunk_size = Chunk_size_EOF_calc(GB_avail)

In [96]:
compute_EOFs(years, chunk_size)

11725.266805887222
